In [0]:
# dbutils.widgets.text("date_filter_min", "2025-10-01", "Start Date")
# dbutils.widgets.text("date_filter_max", "2025-10-31", "End Date")
dbutils.widgets.text("workspace_id", "1444828305810485", "workspace id")
dbutils.widgets.text("catalog_name", "renjiharold_demo", "Catalog Name")
dbutils.widgets.text("schema_name", "waf_db", "Schema Name")

In [0]:
# Get catalog and schema name from widgets
catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

In [0]:
%sql
CREATE OR REPLACE TABLE waf_performance
AS
WITH PE_01 AS (
SELECT 
    billing_origin_product
  , SUM(runs) AS runs_total
  , SUM(dbu_usage) AS dbu_usage_total
  , 'PE_01' AS waf_id
  , 'Detailed' AS dataset_type
  , SUM(CASE WHEN is_serverless = true THEN runs ELSE 0 END) AS sum_serverless_run
  , SUM(CASE WHEN is_serverless = true THEN dbu_usage ELSE 0 END) AS sum_serverless_dbu
  , SUM(CASE WHEN is_serverless = false THEN runs ELSE 0 END) AS sum_non_serverless_run
  , SUM(CASE WHEN is_serverless = false THEN dbu_usage ELSE 0 END) AS sum_non_serverless_dbu
  , SUM(CASE WHEN is_serverless = true THEN runs ELSE 0 END)/SUM(runs) AS pct_serverless_runs
  , SUM(CASE WHEN is_serverless = true THEN dbu_usage ELSE 0 END)/SUM(dbu_usage) AS pct_serverless_dbu
FROM 
(SELECT 
   COUNT(*) AS runs
  ,SUM(usage_quantity) AS dbu_usage, CASE WHEN billing_origin_product = 'ALL_PURPOSE' THEN 'INTERACTIVE' ELSE billing_origin_product END AS billing_origin_product
  , CASE WHEN sku_name LIKE '%SERVERLESS%' OR product_features.is_serverless = true THEN true ELSE false END AS is_serverless
 FROM system.billing.usage 
--  WHERE usage_date BETWEEN :date_filter_min AND :date_filter_max
 WHERE usage_date >= date_add(current_date(), -90)
--  AND array_contains(:workspace_id,workspace_id)
 AND workspace_id = :workspace_id
 AND usage_unit = 'DBU'
 AND billing_origin_product IN ('JOBS','MODEL_SERVING','LAKEFLOW_CONNECT','SQL','INTERACTIVE','DLT','ALL_PURPOSE')
 GROUP BY CASE WHEN billing_origin_product = 'ALL_PURPOSE' THEN 'INTERACTIVE' ELSE billing_origin_product END, CASE WHEN sku_name LIKE '%SERVERLESS%' OR product_features.is_serverless = true THEN true ELSE false END 
)CTE
GROUP BY
billing_origin_product 
),
PE_01_01_perc AS (
  SELECT 
    ROUND((SUM(sum_serverless_dbu)/SUM(dbu_usage_total))*100,2) AS actual_perc,
    'PE-01-01' AS waf_id,
    'Percentage' AS dataset_type
  FROM PE_01
  WHERE billing_origin_product <> 'MODEL_SERVING'
),
-- WAF_PE-02-*_cluster_metrics
usage AS (    
  SELECT usage_metadata.cluster_id AS cluster_id, account_id, workspace_id
  , count(*) as runs, SUM(usage_quantity) AS dbu_usage FROM system.billing.usage 
    -- WHERE usage_date BETWEEN :date_filter_min AND :date_filter_max
     WHERE usage_date >= date_add(current_date(), -90)
    --  AND array_contains(:workspace_id,workspace_id)
    AND workspace_id = :workspace_id
    AND usage_metadata.cluster_id IS NOT NULL
    GROUP BY account_id, workspace_id, usage_metadata.cluster_id
),
compute_met AS (
  SELECT * FROM 
    (select  row_number() over(partition by account_id, workspace_id, cluster_id order by change_time desc) AS rn
        , account_id, workspace_id, c.cluster_id, c.cluster_name, c.worker_node_type, worker_count, max_autoscale_workers, min_autoscale_workers
    from system.compute.clusters c 
    -- WHERE array_contains(:workspace_id,workspace_id)
    WHERE workspace_id = :workspace_id
    AND change_date <= date_add(current_date(), -90)) --TBD to check this filter
    WHERE rn = 1
),
PE_02 AS (
  SELECT 
       *
      , row_number() over( order by dbu_usage desc) AS rank
      , 'PE_02' AS waf_id
      , 'Detailed' AS dataset_type
    FROM(
      SELECT SUM(u.dbu_usage) AS dbu_usage, SUM(u.runs) AS runs, c.cluster_id, c.cluster_name, c.worker_node_type
            ,CASE WHEN ifnull(worker_count,ifnull(max_autoscale_workers,0)) > 1 THEN 'Multi-Node' ELSE 'Single-Node' END AS is_multi_worker
            , ifnull(worker_count,ifnull(max_autoscale_workers,0)) AS max_worker_count
            , CASE WHEN ifnull(c.min_autoscale_workers,0) = ifnull(c.max_autoscale_workers,0) THEN 0 ELSE 1 END AS is_autoscaling
      FROM
      usage u
      INNER JOIN
      compute_met c
      ON u.cluster_id = c.cluster_id
      AND u.account_id = c.account_id
      AND u.workspace_id = c.workspace_id
      GROUP BY
      c.cluster_id, c.cluster_name, c.worker_node_type
      ,CASE WHEN ifnull(worker_count,ifnull(max_autoscale_workers,0)) > 1 THEN 'Multi-Node' ELSE 'Single-Node' END
      , ifnull(worker_count,ifnull(max_autoscale_workers,0)) 
      , CASE WHEN ifnull(c.min_autoscale_workers,0) = ifnull(c.max_autoscale_workers,0) THEN 0 ELSE 1 END 
    )
),
PE_02_02_perc AS (
  SELECT 
    ROUND((SUM(CASE WHEN is_multi_worker = 'Multi-Node' THEN 1 ELSE 0 END)/COUNT(*))*100,2) AS actual_perc,
    'PE-02-02' AS waf_id
    , 'Percentage' AS dataset_type
  FROM PE_02
),
PE_02_04_perc AS (
  SELECT 
    ROUND((SUM(CASE WHEN max_worker_count > 3 THEN 1 ELSE 0 END)/COUNT(*))*100,2) AS actual_perc,
    'PE-02-04' AS waf_id
    , 'Percentage' AS dataset_type
  FROM PE_02
),
PE_02_05 AS (
  SELECT 
    'PE_02_05' AS waf_id,
    'Detailed' AS dataset_type,
    COUNT(*) as count_routines
  FROM system.information_schema.routines WHERE external_language = 'Python' 
),

PE_02_06 AS (
  SELECT 
    'PE_02_06' AS waf_id
  , 'Detailed' AS dataset_type
  , billing_origin_product
  , SUM(runs) AS runs_total
  , SUM(dbu_usage) AS dbu_usage_total
  , SUM(CASE WHEN is_photon = true THEN runs ELSE 0 END) AS sum_photon_run
  , SUM(CASE WHEN is_photon = true THEN dbu_usage ELSE 0 END) AS sum_photon_dbu
  , SUM(CASE WHEN is_photon = false THEN runs ELSE 0 END) AS sum_non_photon_run
  , SUM(CASE WHEN is_photon = false THEN dbu_usage ELSE 0 END) AS sum_non_photon_dbu
  , SUM(CASE WHEN is_photon = true THEN runs ELSE 0 END)/SUM(runs) AS pct_photon_runs
  , SUM(CASE WHEN is_photon = true THEN dbu_usage ELSE 0 END)/SUM(dbu_usage) AS pct_photon_dbu
  FROM (
    SELECT COUNT(*) AS runs,SUM(usage_quantity) AS dbu_usage, billing_origin_product
    , CASE WHEN sku_name LIKE '%PHOTON%' OR product_features.is_photon = true THEN true ELSE false END AS is_photon
    FROM system.billing.usage 
    -- WHERE usage_date BETWEEN :date_filter_min AND :date_filter_max
    WHERE usage_date >= date_add(current_date(), -90)
    -- AND array_contains(:workspace_id,workspace_id)
    AND workspace_id = :workspace_id
    AND billing_origin_product  IN ('JOBS','LAKEFLOW_CONNECT','VECTOR_SEARCH','DATABASE','DLT','ALL_PURPOSE','ONLINE_TABLES','INTERACTIVE')
    AND usage_unit = 'DBU'
    GROUP BY billing_origin_product, CASE WHEN sku_name LIKE '%PHOTON%' OR product_features.is_photon = true THEN true ELSE false END 
)
GROUP BY
billing_origin_product 
),
PE_02_06_perc AS 
(
  SELECT 
    ROUND((SUM(sum_photon_dbu)/SUM(dbu_usage_total))*100,2) AS actual_perc,
    'PE-02-06' AS waf_id,
    'Percentage' AS dataset_type
  FROM PE_02_06
),
PE_02_07_perc AS 
(
  SELECT 
    ROUND((COUNT(DISTINCT split(worker_node_type,'[.]')[0])/COUNT(1))*100,2) AS actual_perc,
    'PE-02-07' AS waf_id,
    'Percentage' AS dataset_type
  FROM PE_02
),


waf_status AS(
SELECT
  waf.waf_id,
  principle,
  best_practice,
  COALESCE(p11p.actual_perc,p22p.actual_perc,p24p.actual_perc,p26p.actual_perc,p27p.actual_perc,0.00) AS actual_perc,
  CASE 
    WHEN waf.waf_id = 'PE-01-02' AND EXISTS (
      SELECT 1 FROM PE_01 WHERE billing_origin_product = 'MODEL_SERVING' LIMIT 1
    ) THEN 'Yes' --TBD dbu usage limit?
    WHEN waf.waf_id = 'PE-02-05' AND NOT EXISTS (
      SELECT 1 FROM PE_02_05 LIMIT 1 
    ) THEN 'Yes'
    WHEN waf.waf_id NOT IN ('PE-01-02','PE-02-05') 
          AND  required_percentage <= COALESCE(p11p.actual_perc,p22p.actual_perc,p24p.actual_perc,p26p.actual_perc,p27p.actual_perc,0) THEN 'Yes'
    ELSE 'No'
  END AS implemented,
  required_percentage
FROM (
  SELECT * FROM VALUES
    ('PE-01-01', 'Utilize serverless capabilities', 'Use serverless architecture',30),
    ('PE-01-02', 'Utilize serverless capabilities', 'Use an enterprise grade model serving service',0),
    --('PE-02-01', 'Utilize serverless capabilities', 'Understand your data ingestion and access patterns'),
    ('PE-02-02', 'Design workloads for performance', 'Use parallel computation where it is beneficial',30),
    --('PE-02-03', 'Design workloads for performance', 'Analyze the whole chain of execution'),
    ('PE-02-04', 'Design workloads for performance', 'Prefer larger clusters',30),
    ('PE-02-05', 'Design workloads for performance', 'Use native Spark operations',0),
    ('PE-02-06', 'Design workloads for performance', 'Use native platform engines',30),
    ('PE-02-07', 'Design workloads for performance', 'Understand your hardware and workload type',20)
    --('PE-02-08', 'Design workloads for performance', 'Use caching'),
    --('PE-02-09', 'Design workloads for performance', 'Use compaction'),
) waf(waf_id, principle, best_practice,required_percentage)
LEFT JOIN 
PE_01_01_perc p11p
ON waf.waf_id = p11p.waf_id
LEFT JOIN 
PE_02_02_perc p22p
ON waf.waf_id = p22p.waf_id
LEFT JOIN
PE_02_04_perc p24p
ON waf.waf_id = p24p.waf_id
LEFT JOIN
PE_02_06_perc p26p
ON waf.waf_id = p26p.waf_id
LEFT JOIN
PE_02_07_perc p27p
ON waf.waf_id = p27p.waf_id
)
SELECT
  waf_id,
  principle,
  best_practice,
  implemented,
  actual_perc,
  required_percentage,
  COUNT(*) OVER (PARTITION BY principle) AS total_controls,
  ROW_NUMBER() OVER (PARTITION BY principle ORDER BY waf_id) AS row_num,
  SUM(CASE WHEN implemented = 'Yes' THEN 1 ELSE 0 END) OVER (PARTITION BY principle) AS implemented_controls,
  ROUND(100 * SUM(CASE WHEN implemented = 'Yes' THEN 1 ELSE 0 END) OVER (PARTITION BY principle)/ COUNT(*) OVER (PARTITION BY principle) , 0) AS completion_percent,
  ROUND(100 * SUM(CASE WHEN implemented = 'Yes' THEN 1 ELSE 0 END) OVER () / COUNT(*) OVER (), 0) AS total_percentage,
  'Summary' AS dataset_type,
  '' AS billing_origin_product,
  0 AS runs_total,
  0 AS dbu_usage_total, 
  0 AS sum_serverless_run,
  0 AS sum_serverless_dbu,
  0 AS sum_non_serverless_run,
  0 AS sum_non_serverless_dbu,
  0 AS pct_serverless_runs,
  0 AS pct_serverless_dbu,
  '' AS cluster_id,
  '' AS cluster_name,
  '' AS worker_node_type,
  '' AS is_multi_worker,
  0 AS max_worker_count,
  NULL AS is_autoscaling,
  '' AS rank,
  0 AS count_routines,
  0 AS sum_photon_run,
  0 AS sum_photon_dbu,
  0 AS sum_non_photon_run,
  0 AS sum_non_photon_dbu,
  0 AS pct_photon_runs,
  0 AS pct_photon_dbu
FROM waf_status
UNION ALL
SELECT
  waf_id,
  '' AS principle,
  '' AS best_practice,
  '' AS implemented,
  0 AS actual_perc,
  0 AS required_percentage,
  0 AS total_controls,
  0 AS row_num,
  0 AS implemented_controls,
  0 AS completion_percent,
  0 AS total_percentage,
  dataset_type,
  billing_origin_product,
  runs_total,
  dbu_usage_total, 
  sum_serverless_run,
  sum_serverless_dbu,
  sum_non_serverless_run,
  sum_non_serverless_dbu,
  pct_serverless_runs,
  pct_serverless_dbu,
  '' AS cluster_id,
  '' AS cluster_name,
  '' AS worker_node_type,
  '' AS is_multi_worker,
  0 AS max_worker_count,
  NULL AS is_autoscaling,
  '' AS rank,
  0 AS count_routines,
  0 AS sum_photon_run,
  0 AS sum_photon_dbu,
  0 AS sum_non_photon_run,
  0 AS sum_non_photon_dbu,
  0 AS pct_photon_runs,
  0 AS pct_photon_dbu
FROM PE_01
UNION ALL 
SELECT
  waf_id,
  '' AS principle,
  '' AS best_practice,
  '' AS implemented,
  0 AS actual_perc,
  0 AS required_percentage,
  0 AS total_controls,
  0 AS row_num,
  0 AS implemented_controls,
  0 AS completion_percent,
  0 AS total_percentage,
  dataset_type,
  '' AS billing_origin_product,
  runs AS runs_total,
  dbu_usage AS dbu_usage_total, 
  0 AS sum_serverless_run,
  0 AS sum_serverless_dbu,
  0 AS sum_non_serverless_run,
  0 AS sum_non_serverless_dbu,
  0 AS pct_serverless_runs,
  0 AS pct_serverless_dbu,
  cluster_id,
  cluster_name,
  worker_node_type,
  is_multi_worker,
  max_worker_count,
  is_autoscaling,
  CASE WHEN rank <= 10 THEN 'Top 10' ELSE 'Others' END AS rank,
  0 AS count_routines,
  0 AS sum_photon_run,
  0 AS sum_photon_dbu,
  0 AS sum_non_photon_run,
  0 AS sum_non_photon_dbu,
  0 AS pct_photon_runs,
  0 AS pct_photon_dbu
FROM PE_02
UNION ALL
SELECT
  waf_id,
  '' AS principle,
  '' AS best_practice,
  '' AS implemented,
  0 AS actual_perc,
  0 AS required_percentage,
  0 AS total_controls,
  0 AS row_num,
  0 AS implemented_controls,
  0 AS completion_percent,
  0 AS total_percentage,
  dataset_type,
  '' AS billing_origin_product,
  0 AS runs_total,
  0 AS dbu_usage_total, 
  0 AS sum_serverless_run,
  0 AS sum_serverless_dbu,
  0 AS sum_non_serverless_run,
  0 AS sum_non_serverless_dbu,
  0 AS pct_serverless_runs,
  0 AS pct_serverless_dbu,
  '' AS cluster_id,
  '' AS cluster_name,
  '' AS worker_node_type,
  '' AS is_multi_worker,
  0 AS max_worker_count,
  NULL AS is_autoscaling,
  '' AS rank,
  count_routines,
  0 AS sum_photon_run,
  0 AS sum_photon_dbu,
  0 AS sum_non_photon_run,
  0 AS sum_non_photon_dbu,
  0 AS pct_photon_runs,
  0 AS pct_photon_dbu
FROM PE_02_05
UNION ALL
SELECT
  waf_id,
  '' AS principle,
  '' AS best_practice,
  '' AS implemented,
  0 AS actual_perc,
  0 AS required_percentage,
  0 AS total_controls,
  0 AS row_num,
  0 AS implemented_controls,
  0 AS completion_percent,
  0 AS total_percentage,
  dataset_type,
  billing_origin_product,
  runs_total,
  dbu_usage_total, 
  0 AS sum_serverless_run,
  0 AS sum_serverless_dbu,
  0 AS sum_non_serverless_run,
  0 AS sum_non_serverless_dbu,
  0 AS pct_serverless_runs,
  0 AS pct_serverless_dbu,
  '' AS cluster_id,
  '' AS cluster_name,
  '' AS worker_node_type,
  '' AS is_multi_worker,
  0 AS max_worker_count,
  NULL AS is_autoscaling,
  '' AS rank,
  0 AS count_routines,
  sum_photon_run,
  sum_photon_dbu,
  sum_non_photon_run,
  sum_non_photon_dbu,
  pct_photon_runs,
  pct_photon_dbu
FROM PE_02_06
ORDER BY waf_id

In [0]:
%sql
INSERT INTO waf_log (run_date, current_percent, prev_percent, percent_change, pillar)
SELECT
  current_timestamp(),
  total_percentage,
  prev_percent,
  CASE
    WHEN prev_percent IS NULL OR prev_percent = 0 THEN 0
    ELSE ROUND((total_percentage - prev_percent) / prev_percent * 100, 2)
  END AS percent_change,
  'performance'
FROM (
  SELECT
    total_percentage,
    (SELECT 
      current_percent
     FROM waf_log
     WHERE pillar = 'performance'
     ORDER BY run_id DESC
     LIMIT 1) AS prev_percent
  FROM waf_performance
  WHERE dataset_type = 'Summary'
  LIMIT 1
);